In [ ]:
!pip install -q opencv-python-headless pytube
!pip install -q --upgrade pytube


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.9 MB/s eta 0:00:00


In [ ]:
from pytube import YouTube

# YouTube deepfake sample video
video_url = 'https://www.youtube.com/watch?v=ZJrffEfCMrs'

yt = YouTube(video_url)
# Try to extract video information
try:
    stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    stream.download(filename='deepfake_sample.mp4')
    print("Video downloaded successfully as 'deepfake_sample.mp4'")
except Exception as e:
    print(f"An error occurred: {e}")
    print("This is often caused by changes in YouTube's structure or the video's availability.")
    print("You may need to try a different video URL or wait for pytube to be updated.")

An error occurred: HTTP Error 400: Bad Request
This is often caused by changes in YouTube's structure or the video's availability.
You may need to try a different video URL or wait for pytube to be updated.


In [ ]:
from google.colab import files
import cv2
import os

# Upload video file
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

cap = cv2.VideoCapture(video_path)

# Check if video opened correctly
if not cap.isOpened():
    print("Error: Cannot open video.")
else:
    print("Video opened successfully.")

# Create directory
if not os.path.exists("frames"):
    os.makedirs("frames")

frame_count = 0
saved_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Save every 10th frame
    if frame_count % 10 == 0:
        cv2.imwrite(f"frames/frame_{frame_count}.jpg", frame)
        saved_count += 1
    frame_count += 1

cap.release()
print(f"Total frames: {frame_count}, Saved frames: {saved_count}")

Saving videoplayback.mp4 to videoplayback.mp4
Video opened successfully.
Total frames: 2785, Saved frames: 279


In [ ]:
from keras.applications.xception import Xception, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing import image
import numpy as np

# Build model
base_model = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# NOTE: You need a pretrained deepfake detection weight file. This is a placeholder.
# If you have a model file, use model.load_weights('your_model.h5')
print("Model structure loaded. Now load your weights if available.")


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Model structure loaded. Now load your weights if available.


In [ ]:
from keras.applications.xception import Xception, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing import image
import numpy as np

# Build model
base_model = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# NOTE: You need a pretrained deepfake detection weight file. This is a placeholder.
# If you have a model file, use model.load_weights('your_model.h5')
print("Model structure loaded. Now load your weights if available.")


Model structure loaded. Now load your weights if available.


In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_and_preprocess_face(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    if len(faces) == 0:
        return None

    x, y, w, h = faces[0]
    face_img = img[y:y+h, x:x+w]
    face_img = cv2.resize(face_img, (299, 299))
    face_array = image.img_to_array(face_img)
    face_array = np.expand_dims(face_array, axis=0)
    return preprocess_input(face_array)
from tqdm import tqdm

predictions = []

for img_file in tqdm(sorted(os.listdir("frames"))):
    img_path = os.path.join("frames", img_file)
    face_input = detect_and_preprocess_face(img_path)

    if face_input is not None:
        pred = model.predict(face_input)[0][0]
        predictions.append(pred)

print(f"Predictions: {predictions}")


  0%|          | 0/279 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


  1%|▏         | 4/279 [00:02<03:10,  1.45it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 401ms/step


  3%|▎         | 8/279 [00:03<01:35,  2.85it/s]

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 382ms/step


100%|█████████▉| 278/279 [01:51<00:00,  2.41it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 401ms/step


100%|██████████| 279/279 [01:52<00:00,  2.48it/s]

Predictions: [np.float32(0.56620806), np.float32(0.54753786), np.float32(0.56257534), np.float32(0.55844957), np.float32(0.5276647), np.float32(0.5999981), np.float32(0.53203297), np.float32(0.5145216), np.float32(0.5424436), np.float32(0.5568538), np.float32(0.55343306), np.float32(0.545186), np.float32(0.5520791), np.float32(0.56725436), np.float32(0.5547376), np.float32(0.5356398), np.float32(0.573365), np.float32(0.57748175), np.float32(0.5359044), np.float32(0.52484375), np.float32(0.51603854), np.float32(0.5939561), np.float32(0.5654916), np.float32(0.54430616), np.float32(0.5587198), np.float32(0.537401), np.float32(0.5493647), np.float32(0.5612173), np.float32(0.56499976), np.float32(0.5701217), np.float32(0.5830485), np.float32(0.57239383), np.float32(0.5578541), np.float32(0.5696593), np.float32(0.5494182), np.float32(0.5628145), np.float32(0.54302645), np.float32(0.5517657), np.float32(0.56491685), np.float32(0.57991916), np.float32(0.5715102), np.float32(0.5459316), np.floa

In [ ]:
import numpy as np

if len(predictions) == 0:
    print("No faces detected.")
else:
    avg_score = np.mean(predictions)
    print(f"\nAverage deepfake probability: {avg_score:.4f}")

    if avg_score >= 0.52:
        print("🔴 This video is likely a Deepfake.")
    else:
        print("🟢 This video appears to be Real.")



Average deepfake probability: 0.5646
🔴 This video is likely a Deepfake.


SyntaxError: invalid syntax (<ipython-input-7-f188d5ac1197>, line 1)